# MARATONA BEHIND THE CODE 2020

## DESAFIO 6 - LIT

<hr>

## Download dos conjuntos de dados em formato .csv

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', 25)

In [3]:
dataset = pd.read_csv('training_dataset.csv')
dataset.tail()

,id,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,importante_ter_certificado,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados,categoria
15852,620397030.0,Bacharelado,UFF,Outros,Borracha,0.0,0.0,1.0,8.0,NaN,10.0,NaN,NaN,NaN,perfil6
15853,229931283.0,Bacharelado,NaN,Advogado,NaN,0.0,0.0,1.0,7.0,Linkedin,42.0,17.0,15.0,NaN,perfil5
15854,480674907.0,Tecnólogo,UNIP,Sócio/Dono/Proprietário,NaN,0.0,NaN,1.0,7.0,Outros,30.0,9.0,8.0,0.0,perfil5
15855,559626807.0,SEM FORMAÇÃO,UNIVERSIDADE NOVE DE JULHO,Advogado,Estado,0.0,0.0,1.0,10.0,NaN,226.0,102.0,93.0,1.0,perfil1
15856,743652801.0,NaN,FGV-RJ,NaN,Siderurgica,1.0,1.0,1.0,9.0,Outros,125.0,98.0,97.0,1.0,perfil1


Sobre o arquivo "training_dataset.csv", temos algumas informações gerais sobre os usuários da plataforma:

**id**

**graduacao**

**universidade**

**profissao**

**organizacao**

**pretende_fazer_cursos_lit**

**interesse_mba_lit**

**importante_ter_certificado**

**horas_semanais_estudo**

**como_conheceu_lit**

**total_modulos**

**modulos_iniciados**

**modulos_finalizados**

**certificados**

**categoria**

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15857 entries, 0 to 15856
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          13945 non-null  float64
 1   graduacao                   13950 non-null  object 
 2   universidade                13920 non-null  object 
 3   profissao                   13977 non-null  object 
 4   organizacao                 13961 non-null  object 
 5   pretende_fazer_cursos_lit   13989 non-null  float64
 6   interesse_mba_lit           14003 non-null  float64
 7   importante_ter_certificado  13918 non-null  float64
 8   horas_semanais_estudo       13959 non-null  float64
 9   como_conheceu_lit           13915 non-null  object 
 10  total_modulos               13987 non-null  float64
 11  modulos_iniciados           14044 non-null  float64
 12  modulos_finalizados         13924 non-null  float64
 13  certificados                139

In [5]:
dataset.nunique()

id                            13945
graduacao                         6
universidade                     21
profissao                        12
organizacao                      11
pretende_fazer_cursos_lit         2
interesse_mba_lit                 2
importante_ter_certificado        1
horas_semanais_estudo             9
como_conheceu_lit                 9
total_modulos                   578
modulos_iniciados               372
modulos_finalizados             339
certificados                     23
categoria                         6
dtype: int64

<hr>

## Detalhamento do desafio: classificação multiclasse

Este é um desafio cujo objetivo de negócio é a segmentação dos usuários de uma plataforma de ensino. Para tal, podemos utilizar duas abordagens: aprendizado de máquina supervisionado (classificação) ou não-supervisionado (clustering). Neste desafio será aplicada a classificação, pois é disponível um dataset já com "labels", ou em outras palavras, já com exemplos de dados juntamente com a variável alvo.

Na biblioteca scikit-learn temos diversos algoritmos para classificação. O participante é livre para utilizar o framework que desejar para completar esse desafio.

Neste notebook será mostrado um exeplo de uso do algoritmo "Decision Tree" para classificar parte dos estudantes em seis diferentes perfís.

# Divisão do banco

A coluna-alvo neste desafio é a coluna ``categoria``

<hr>

In [6]:

from sklearn.model_selection import StratifiedShuffleSplit

In [7]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

In [8]:
for train_index, val_index in split.split(dataset, dataset.categoria):
    data_train = dataset.loc[train_index]    
    data_val = dataset.loc[val_index]

In [9]:
print(data_train.shape)
print(data_val.shape)

(12685, 15)
(3172, 15)


# Pre-processando o dataset antes do treinamento

### Removendo todas as linhas que possuem algum valor nulos em determinadas colunas

Usando o método Pandas **DataFrame.dropna()** você pode remover todas as linhas nulas do dataset.

Docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html

In [10]:
dd = data_train.dropna()

In [11]:
x_train = dd.drop(['id', 'importante_ter_certificado', 'categoria'], axis=1)

In [12]:
y_train = dd.categoria

In [13]:
x_train['rate1'] = x_train.modulos_finalizados/x_train.modulos_iniciados
x_train['rate2'] = x_train.certificados /x_train.modulos_iniciados
x_train['rate3'] = x_train.modulos_finalizados / x_train.total_modulos
x_train['rate4'] = x_train.certificados/x_train.total_modulos
x_train['rate5'] = x_train.modulos_iniciados/x_train.total_modulos
x_train['rate6'] = x_train.certificados /x_train.modulos_finalizados
x_train['rate7'] = x_train.certificados /x_train.total_modulos

x_train['mul1'] = x_train.certificados * x_train.modulos_finalizados

x_train['add1'] = x_train.certificados + x_train.modulos_finalizados

In [14]:
x_train.tail()

,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados,rate1,rate2,rate3,rate4,rate5,rate6,rate7,mul1,add1
9332,Tecnólogo,Escola Paulista de Direito,Analista Senior,Laminados,0.0,0.0,7.0,Mídia (revista/jornal/web),100.0,99.0,99.0,3.0,1.000000,0.030303,0.990000,0.03,0.990000,0.030303,0.03,297.0,102.0
2945,Tecnólogo,UNIVERSIDADE NOVE DE JULHO,Sócio/Dono/Proprietário,e-commerce,1.0,1.0,10.0,Minha empresa - benefício LITpass,29.0,5.0,3.0,0.0,0.600000,0.000000,0.103448,0.00,0.172414,0.000000,0.00,0.0,3.0
5659,Tecnólogo,UFF,Supervisor,Entretenimento,0.0,0.0,4.0,Mídia (revista/jornal/web),31.0,11.0,10.0,0.0,0.909091,0.000000,0.322581,0.00,0.354839,0.000000,0.00,0.0,10.0
1195,SEM FORMAÇÃO,USP,Analista,e-commerce,1.0,1.0,7.0,Outros,33.0,4.0,2.0,0.0,0.500000,0.000000,0.060606,0.00,0.121212,0.000000,0.00,0.0,2.0
1863,Licenciatura,UNIVERSIDADE NOVE DE JULHO,Diretor,Entretenimento,1.0,1.0,10.0,Saint Paul,37.0,4.0,1.0,0.0,0.250000,0.000000,0.027027,0.00,0.108108,0.000000,0.00,0.0,1.0


In [15]:
from sklearn.preprocessing import OrdinalEncoder

In [16]:
cat_cols = list(x_train.select_dtypes(include=['object']).columns)
cat_cols

['graduacao', 'universidade', 'profissao', 'organizacao', 'como_conheceu_lit']

In [17]:
ordinal = OrdinalEncoder()

In [18]:
xx = x_train.copy()

In [19]:
aa = pd.DataFrame( ordinal.fit_transform(x_train[cat_cols]))
aa.index = x_train.index

xx[cat_cols] = aa

In [20]:
xx.tail()

,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados,rate1,rate2,rate3,rate4,rate5,rate6,rate7,mul1,add1
9332,5.0,1.0,2.0,5.0,0.0,0.0,7.0,5.0,100.0,99.0,99.0,3.0,1.000000,0.030303,0.990000,0.03,0.990000,0.030303,0.03,297.0,102.0
2945,5.0,16.0,11.0,9.0,1.0,1.0,10.0,4.0,29.0,5.0,3.0,0.0,0.600000,0.000000,0.103448,0.00,0.172414,0.000000,0.00,0.0,3.0
5659,5.0,7.0,10.0,3.0,0.0,0.0,4.0,5.0,31.0,11.0,10.0,0.0,0.909091,0.000000,0.322581,0.00,0.354839,0.000000,0.00,0.0,10.0
1195,4.0,18.0,1.0,9.0,1.0,1.0,7.0,6.0,33.0,4.0,2.0,0.0,0.500000,0.000000,0.060606,0.00,0.121212,0.000000,0.00,0.0,2.0
1863,2.0,16.0,5.0,3.0,1.0,1.0,10.0,7.0,37.0,4.0,1.0,0.0,0.250000,0.000000,0.027027,0.00,0.108108,0.000000,0.00,0.0,1.0


In [21]:
x_train = xx

### Scaler

In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
scaler = StandardScaler()

In [24]:
#x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)

## Repetindo no banco de validação

In [25]:
dd = data_val.dropna()
x_val = dd.drop(['id', 'importante_ter_certificado', 'categoria'], axis=1)
y_val = dd.categoria

In [26]:
x_val['rate1'] = x_val.modulos_finalizados/x_val.modulos_iniciados
x_val['rate2'] = x_val.certificados /x_val.modulos_iniciados
x_val['rate3'] = x_val.modulos_finalizados / x_val.total_modulos
x_val['rate4'] = x_val.certificados/x_val.total_modulos
x_val['rate5'] = x_val.modulos_iniciados/x_val.total_modulos
x_val['rate6'] = x_val.certificados /x_val.modulos_finalizados
x_val['rate7'] = x_val.certificados /x_val.total_modulos

x_val['mul1'] = x_val.certificados * x_val.modulos_finalizados

x_val['add1'] = x_val.certificados + x_val.modulos_finalizados

In [27]:
aa = pd.DataFrame( ordinal.transform(x_val[cat_cols]))
aa.index = x_val.index

x_val[cat_cols] = aa

In [28]:
x_val.head()

,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados,rate1,rate2,rate3,rate4,rate5,rate6,rate7,mul1,add1
15745,3.0,9.0,9.0,7.0,1.0,1.0,7.0,1.0,31.0,8.0,5.0,0.0,0.625000,0.000000,0.161290,0.000000,0.258065,0.000000,0.000000,0.0,5.0
9322,1.0,11.0,4.0,7.0,1.0,1.0,9.0,2.0,135.0,47.0,39.0,1.0,0.829787,0.021277,0.288889,0.007407,0.348148,0.025641,0.007407,39.0,40.0
7429,1.0,13.0,6.0,3.0,1.0,1.0,7.0,3.0,33.0,3.0,1.0,0.0,0.333333,0.000000,0.030303,0.000000,0.090909,0.000000,0.000000,0.0,1.0
7790,1.0,3.0,5.0,7.0,1.0,1.0,10.0,2.0,239.0,42.0,25.0,0.0,0.595238,0.000000,0.104603,0.000000,0.175732,0.000000,0.000000,0.0,25.0
5696,2.0,4.0,0.0,1.0,1.0,1.0,7.0,4.0,57.0,11.0,7.0,0.0,0.636364,0.000000,0.122807,0.000000,0.192982,0.000000,0.000000,0.0,7.0


In [29]:
#x_val = pd.DataFrame(scaler.transform(x_val), columns=x_val.columns)

# Treinamento

In [30]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [41]:
clf = RandomForestClassifier(max_depth=10)

In [42]:
clf.fit(x_train, y_train)

pred_train = clf.predict(x_train)

In [34]:
print(classification_report(pred_train, y_train))

              precision    recall  f1-score   support

     perfil1       0.85      0.94      0.89       310
     perfil2       0.89      0.86      0.87       591
     perfil3       0.83      0.87      0.85       125
     perfil4       0.89      0.85      0.87       248
     perfil5       0.83      0.85      0.84       245
     perfil6       0.90      0.87      0.88       614

    accuracy                           0.87      2133
   macro avg       0.86      0.87      0.87      2133
weighted avg       0.87      0.87      0.87      2133



# validação

In [35]:
pred_val = clf.predict(x_val)

In [36]:

print(classification_report(y_val, pred_val))

              precision    recall  f1-score   support

     perfil1       0.82      0.87      0.84        75
     perfil2       0.81      0.82      0.81       151
     perfil3       0.84      0.76      0.80        42
     perfil4       0.85      0.81      0.83        69
     perfil5       0.88      0.89      0.88        79
     perfil6       0.86      0.85      0.86       142

    accuracy                           0.84       558
   macro avg       0.84      0.83      0.84       558
weighted avg       0.84      0.84      0.84       558



In [37]:
print(accuracy_score(y_val, pred_val))

0.8387096774193549


## Insights
SelectKBest

In [38]:

from sklearn.feature_selection import SelectKBest

In [39]:

for i in range(x_train.shape[1], 5, -1):
    skb = SelectKBest(k = i)
    xx = skb.fit_transform(x_train, y_train)
    clf.fit(xx, y_train)
    pred = clf.predict(skb.transform(x_val))
    print(i, accuracy_score(y_val, pred))

21 0.8369175627240143
20 0.8369175627240143
19 0.8369175627240143
18 0.8351254480286738
17 0.8333333333333334
16 0.8315412186379928
15 0.8315412186379928
14 0.8279569892473119
13 0.8225806451612904
12 0.8279569892473119
11 0.8189964157706093
10 0.8261648745519713
9 0.8225806451612904
8 0.8243727598566308
7 0.8225806451612904
6 0.589605734767025


In [40]:
for i in range(1,30, 2):
    clf2 = RandomForestClassifier(max_depth=i)
    clf2.fit(x_train, y_train)
    pred = clf2.predict(x_val)
    print(i, accuracy_score(y_val, pred))

1 0.5734767025089605
3 0.7544802867383512
5 0.8189964157706093
7 0.8387096774193549
9 0.8387096774193549
11 0.8387096774193549
13 0.8387096774193549
15 0.8369175627240143
17 0.8369175627240143
19 0.8297491039426523
21 0.8351254480286738
23 0.8351254480286738
25 0.8369175627240143
27 0.8387096774193549
29 0.8369175627240143


<hr>

# Scoring dos dados necessários para entregar a solução

Como entrega da sua solução, esperamos os resultados classificados no seguinte dataset chamado "to_be_scored.csv":

## dataset de teste

In [43]:
dataset_test = pd.read_csv('to_be_scored.csv')
dataset_test.tail()

,id,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,importante_ter_certificado,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados
995,741745189,SEM FORMAÇÃO,UNIVERSIDADE NOVE DE JULHO,Coordenador,Borracha,0,0,1,2,YouTube,245,63,45,1
996,996387882,Bacharelado,UFRN,Assessor,Estado,1,1,1,8,Google,163,111,96,0
997,668869418,Bacharelado,UFSCar,Gerente,Siderurgica,1,1,1,9,Facebook,56,10,4,0
998,676531718,Tecnólogo,FATEC,Outros,servicos,1,1,1,9,Mídia (revista/jornal/web),138,60,30,0
999,388402340,Especialização,UNIP,Advogado,Montadora,0,0,1,9,Mídia (revista/jornal/web),22,20,19,0


In [45]:
dataset_test.isna().sum()

id                            0
graduacao                     0
universidade                  0
profissao                     0
organizacao                   0
pretende_fazer_cursos_lit     0
interesse_mba_lit             0
importante_ter_certificado    0
horas_semanais_estudo         0
como_conheceu_lit             0
total_modulos                 0
modulos_iniciados             0
modulos_finalizados           0
certificados                  0
dtype: int64

# Atenção!

O dataframe ``to_be_scored`` é a sua "folha de respostas". Note que a coluna "categoria" não existe nessa amostra, que não pode ser então utilizada para treino de modelos de aprendizado supervisionado.

In [46]:
dataset_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   id                          1000 non-null   int64 
 1   graduacao                   1000 non-null   object
 2   universidade                1000 non-null   object
 3   profissao                   1000 non-null   object
 4   organizacao                 1000 non-null   object
 5   pretende_fazer_cursos_lit   1000 non-null   int64 
 6   interesse_mba_lit           1000 non-null   int64 
 7   importante_ter_certificado  1000 non-null   int64 
 8   horas_semanais_estudo       1000 non-null   int64 
 9   como_conheceu_lit           1000 non-null   object
 10  total_modulos               1000 non-null   int64 
 11  modulos_iniciados           1000 non-null   int64 
 12  modulos_finalizados         1000 non-null   int64 
 13  certificados                1000 non-null   int64

<hr>

# Atenção!

# Para poder aplicar seu modelo e classificar a folha de respostas, você precisa primeiro aplicar as mesmas transformações com colunas que você aplicou no dataset de treino.

# Não remova ou adicione linhas na folha de respostas. 

# Não altere a ordem das linhas na folha de respostas.

# Ao final, as 1000 entradas devem estar classificadas, com os valores previstos em uma coluna chamada "target"

<hr>

Na célula abaixo, repetimos rapidamente os mesmos passos de pré-processamento usados no exemplo dado com árvore de decisão

In [47]:
x_test = dataset_test.drop(['id', 'importante_ter_certificado'], axis=1)

In [48]:
x_test['rate1'] = x_test.modulos_finalizados/x_test.modulos_iniciados
x_test['rate2'] = x_test.certificados /x_test.modulos_iniciados
x_test['rate3'] = x_test.modulos_finalizados / x_test.total_modulos
x_test['rate4'] = x_test.certificados/x_test.total_modulos
x_test['rate5'] = x_test.modulos_iniciados/x_test.total_modulos
x_test['rate6'] = x_test.certificados /x_test.modulos_finalizados
x_test['rate7'] = x_test.certificados /x_test.total_modulos

x_test['mul1'] = x_test.certificados * x_test.modulos_finalizados

x_test['add1'] = x_test.certificados + x_test.modulos_finalizados

In [49]:
aa = pd.DataFrame( ordinal.transform(x_test[cat_cols]))
aa.index = x_test.index

x_test[cat_cols] = aa

In [50]:
x_test.head()

,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados,rate1,rate2,rate3,rate4,rate5,rate6,rate7,mul1,add1
0,0.0,19.0,8.0,0.0,0,0,3,7.0,58,3,2,0,0.666667,0.000000,0.034483,0.000000,0.051724,0.000000,0.000000,0,2
1,1.0,12.0,9.0,1.0,1,1,7,5.0,62,19,17,0,0.894737,0.000000,0.274194,0.000000,0.306452,0.000000,0.000000,0,17
2,0.0,12.0,6.0,7.0,0,0,9,1.0,36,3,1,0,0.333333,0.000000,0.027778,0.000000,0.083333,0.000000,0.000000,0,1
3,2.0,12.0,8.0,4.0,1,1,9,1.0,145,61,54,1,0.885246,0.016393,0.372414,0.006897,0.420690,0.018519,0.006897,54,55
4,1.0,2.0,6.0,3.0,0,0,8,4.0,33,33,29,0,0.878788,0.000000,0.878788,0.000000,1.000000,0.000000,0.000000,0,29


# Predição do Teste

In [51]:
y_pred = clf.predict(x_test)
result = x_test
result['target'] = y_pred
result.tail()

,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados,rate1,rate2,rate3,rate4,rate5,rate6,rate7,mul1,add1,target
995,4.0,16.0,4.0,0.0,0,0,2,8.0,245,63,45,1,0.714286,0.015873,0.183673,0.004082,0.257143,0.022222,0.004082,45,46,perfil1
996,0.0,10.0,3.0,4.0,1,1,8,1.0,163,111,96,0,0.864865,0.000000,0.588957,0.000000,0.680982,0.000000,0.000000,0,96,perfil2
997,0.0,11.0,7.0,8.0,1,1,9,0.0,56,10,4,0,0.400000,0.000000,0.071429,0.000000,0.178571,0.000000,0.000000,0,4,perfil6
998,5.0,3.0,8.0,10.0,1,1,9,5.0,138,60,30,0,0.500000,0.000000,0.217391,0.000000,0.434783,0.000000,0.000000,0,30,perfil2
999,1.0,13.0,0.0,6.0,0,0,9,5.0,22,20,19,0,0.950000,0.000000,0.863636,0.000000,0.909091,0.000000,0.000000,0,19,perfil3


### Salvando a folha de respostas como um arquivo .csv para ser submetido

In [54]:
#project.save_data(file_name="results.csv", data=result.to_csv(index=False))
result.to_csv('result.csv', index=False)

# Atenção

# A execução da célula acima irá criar um novo "data asset" no seu projeto no Watson Studio. Você precisará realizar o download deste arquivo juntamente com este notebook e criar um arquivo zip com os arquivos **results.csv** e **notebook.ipynb** para submissão. (os arquivos devem estar nomeados desta forma)

<hr>

## Parabéns!

Se você já está satisfeito com a sua solução, vá até a página abaixo e envie os arquivos necessários para submissão.

# https://lit.maratona.dev
